In [3]:
import cv2
import numpy as np
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras import models
import keras_tuner as kt
from keras_tuner.tuners import Hyperband

In [2]:
def load_data(validation_split):

    # Load data from csv file into data frame, drop all rows that have missing values
    df = read_csv("training.csv")
    print(df["Image"].count())
    df = df.dropna()
    print(df["Image"].count())

    # Convert the rows of the image column from pixel values separated by spaces to numpy arrays
    df["Image"] = df["Image"].apply(lambda img: np.fromstring(img, sep=" "))

    # Create numpy matrix from image column by stacking the rows vertically
    X_data = np.vstack(df["Image"].values)
    # Normalize pixel values to (0, 1) range
    X_data = X_data / 255
    # Convert to float32, which is the default for Keras
    X_data = X_data.astype("float32")
    # Reshape each row from one dimensional arrays to (height, width, num_channels) = (96, 96, 1)
    X_data = X_data.reshape(-1, 96, 96, 1)
    # Extract labels representing the coordinates of facial landmarks
    Y_data = df[df.columns[:-1]].values

    # Normalize coordinates to (0, 1) range
    Y_data = Y_data / 96
    Y_data = Y_data.astype("float32")

    # Shuffle data
    X_data, Y_data = shuffle(X_data, Y_data)

    # Split data into training set and validation set
    split_index = int(X_data.shape[0] * (1 - validation_split))
    X_train = X_data[:split_index]
    Y_train = Y_data[:split_index]
    X_val = X_data[split_index:]
    Y_val = Y_data[split_index:]

    return X_train, Y_train, X_val, Y_val

In [3]:
X_train, Y_train, X_val, Y_val = load_data(validation_split=0.2)

7049
2140


CNN Model

In [5]:
def create_cnn_model(hp):
    """function to to generate a combination of hyperparameter to use in RandomSearch

    Args:
        hp: object of cominations

    Returns:
        _type_: Sequential model
    """
    
    filters = hp.Choice('num_filters', values=[32, 64], default=64)
    units = hp.Int('units', min_value=32, max_value=512, step=32, default=128)
    activation = hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu' )
    rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.01, sampling='LOG', default=0.001)
    
    model = models.Sequential([
        
        Conv2D(filters=filters, kernel_size=(5,5), input_shape=(96, 96, 1), activation=activation, padding="same"),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(filters=filters, kernel_size=(5,5), activation=activation, padding="same"),
        MaxPool2D(pool_size=(2, 2)),
        Conv2D(filters=filters, kernel_size=(5,5), activation=activation, padding="same"),
        Dropout(rate=rate),
        MaxPool2D(pool_size=(2, 2)),
        Flatten(),
        Dense(units=units, activation=activation),
        Dense(30)
    ])
    
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    return model

In [5]:
tuner = Hyperband(
    create_cnn_model,
    max_epochs=10,
    objective='val_loss',
    seed=1,
    executions_per_trial=2,
    directory='hyperband',
    project_name='cifar10'
)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_filters (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
units (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [7]:
tuner.search(X_train, Y_train,  epochs=10, validation_split=0.1)

Trial 30 Complete [00h 04m 52s]
val_loss: 0.0005298343603499234

Best val_loss So Far: 0.0004672050126828253
Total elapsed time: 01h 40m 49s
INFO:tensorflow:Oracle triggered exit


In [8]:
best_cnn_model = tuner.get_best_models(num_models=1)[0]

In [9]:
best_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        102464    
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0

In [10]:
best_cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 64)        102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        102464    
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0

In [11]:
cnn_val_loss = best_cnn_model.evaluate(X_val, Y_val)

14/14 [==============================] - 2s 126ms/step - loss: 4.1400e-04


In [12]:
cnn_val_loss

0.00041399820474907756

Baseline Model


In [13]:
def create_baseline_model():
    model = models.Sequential()
    model.add(Flatten(input_shape=(96, 96, 1)))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(30))
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

In [14]:
baseline_model = create_baseline_model()
baseline_model.fit(X_train, Y_train, batch_size=200, epochs=10, validation_split=0.2)

Epoch 1/10
7/7 [==============================] - 1s 76ms/step - loss: 16.5986 - val_loss: 0.5971
Epoch 2/10
7/7 [==============================] - 0s 60ms/step - loss: 0.3469 - val_loss: 0.2472
Epoch 3/10
7/7 [==============================] - 0s 64ms/step - loss: 0.2346 - val_loss: 0.2154
Epoch 4/10
7/7 [==============================] - 0s 64ms/step - loss: 0.1986 - val_loss: 0.1750
Epoch 5/10
7/7 [==============================] - 0s 60ms/step - loss: 0.1571 - val_loss: 0.1336
Epoch 6/10
7/7 [==============================] - 0s 61ms/step - loss: 0.1175 - val_loss: 0.0975
Epoch 7/10
7/7 [==============================] - 0s 61ms/step - loss: 0.0842 - val_loss: 0.0693
Epoch 8/10
7/7 [==============================] - 0s 61ms/step - loss: 0.0594 - val_loss: 0.0495
Epoch 9/10
7/7 [==============================] - 0s 61ms/step - loss: 0.0428 - val_loss: 0.0361
Epoch 10/10
7/7 [==============================] - 0s 65ms/step - loss: 0.0311 - val_loss: 0.0278


In [15]:
final_val_loss = baseline_model.evaluate(X_val, Y_val)

14/14 [==============================] - 0s 5ms/step - loss: 0.0268


In [16]:
final_val_loss

0.026807500049471855

In [6]:
# Create directory if it does not exist
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

# Save CNN model weights
best_cnn_model.save_weights("saved_models/cnn_model_weights.h5")

# Save CNN model architecture
with open("saved_models/cnn_model_architecture.json", "w") as f:
         f.write(best_cnn_model.to_json())

NameError: name 'best_cnn_model' is not defined

In [22]:
# save baseline model weights
baseline_model.save_weights("saved_models/baseline_model_weights.h5")

# save baseline model architecture
with open("saved_models/baseline_model_architecture.json", "w") as f:
    f.write(baseline_model.to_json())

# Prediction

In [12]:
def extract_landmarks(y_pred, img_size_x, img_size_y):
    landmarks = []
    for i in range(0, len(y_pred), 2):
        landmark_x, landmark_y = y_pred[i] * img_size_x, y_pred[i+1] * img_size_y
        landmarks.append((landmark_x, landmark_y))
    return landmarks

In [13]:
def load_model(model_name: str):
    with open("saved_models/" + model_name + "_model_architecture.json", "r") as f:
        model = models.model_from_json(f.read())
    model.load_weights("saved_models/" + model_name + "_model_weights.h5")
    return model

In [14]:
def save_img_with_landmarks(img, landmarks, filename):
    # Convert image to grayscale
    img_gray = np.mean(img, axis=-1)

    # Plot image and landmarks
    fig, ax = plt.subplots()
    ax.imshow(img_gray, cmap='gray')
    ax.scatter([landmark[0] for landmark in landmarks], 
               [landmark[1] for landmark in landmarks], 
               marker='o', s=10, c='g')

    # Save image
    plt.savefig(filename)
    plt.close()

In [21]:
def sunglasses_filter(model_name:str='cnn', filter_name:str='sunglasses'):

    cap = cv2.VideoCapture(0)
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        # Crop face
        if len(faces) > 0:
            (x,y,w,h) = faces[0]
            gray = gray[y:y+h, x:x+w]
            frame = frame[y:y+h, x:x+w]
        else:
            continue
        
        # Resize face
        gray = cv2.resize(gray, dsize=(96, 96), interpolation=cv2.INTER_AREA)
        
        # Convert frame to RGB format
        orig_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        orig_size_x, orig_size_y = orig_img.shape[0], orig_img.shape[1]

        # Prepare input image
        img = np.expand_dims(gray, axis=2)
        img = img.astype("float32") / 255

        # Predict landmarks
        model = load_model(model_name)
        y_pred = model.predict(np.expand_dims(img, axis=0))[0]
        landmarks = extract_landmarks(y_pred, orig_size_x, orig_size_y)
        
        # Save original image with landmarks on top
        save_img_with_landmarks(orig_img, landmarks, "landmarks.png")
        # Extract x and y values from landmarks of interest
        # eye position
        left_eye_center_x = int(landmarks[0][0])
        left_eye_center_y = int(landmarks[0][1])
        right_eye_center_x = int(landmarks[1][0])
        right_eye_center_y = int(landmarks[1][1])
        left_eye_outer_x = int(landmarks[3][0])
        right_eye_outer_x = int(landmarks[5][0])

        # Load images using PIL
        # PIL has better functions for rotating and pasting compared to cv2
        face_img = Image.fromarray(frame)
        sunglasses_img = Image.open(filter_name + ".png")

        # Resize sunglasses
        sunglasses_width = int((left_eye_outer_x - right_eye_outer_x) * 1.4)
        sunglasses_height = int(sunglasses_img.size[1] * (sunglasses_width / sunglasses_img.size[0]))
        sunglasses_resized = sunglasses_img.resize((sunglasses_width, sunglasses_height))

        # Rotate sunglasses
        eye_angle_radians = np.arctan((right_eye_center_y - left_eye_center_y) / (left_eye_center_x - right_eye_center_x))
        sunglasses_rotated = sunglasses_resized.rotate(np.degrees(eye_angle_radians), expand=True, resample=Image.BICUBIC)

        # Compute positions such that the center of the sunglasses is
        # positioned at the center point between the eyes
        x_offset = int(sunglasses_width * 0.5)
        y_offset = int(sunglasses_height * 0.5)
        pos_x = int((left_eye_center_x + right_eye_center_x) / 2) - x_offset
        pos_y = int((left_eye_center_y + right_eye_center_y) / 2) - y_offset


        # Paste sunglasses on face image
        face_img.paste(sunglasses_rotated, (pos_x, pos_y), sunglasses_rotated)

        # Convert image back to OpenCV format
        final_img = cv2.cvtColor(np.array(face_img), cv2.COLOR_RGB2BGR)

        # Display the resulting image
        cv2.imshow('frame',final_img)
        if cv2.waitKey(1) & 0xFF == ord('o'):
                break
                
    cap.release()
    cv2.destroyAllWindows()

In [22]:
sunglasses_filter()

1/1 [==============================] - 0s 77ms/step


C:\Users\danar\AppData\Local\Temp\ipykernel_9912\517795988.py:63: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  sunglasses_rotated = sunglasses_resized.rotate(np.degrees(eye_angle_radians), expand=True, resample=Image.BICUBIC)


1/1 [==============================] - 0s 87ms/step
